In [ ]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.1 MB/s eta 0:00:00


In [ ]:
!python -m pip install tensorflow

In [ ]:
!nvidia-smi

Fri May 17 06:04:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   36C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/chess_from_pgn_250000.json'

In [ ]:
# pulling in the data
import json

data_path = '/content/drive/MyDrive/chess_from_pgn_250000.json'

with open(data_path, 'r') as file:
    data = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/chess_from_pgn_250000.json'

In [ ]:
total_samples = len(data)
train_end = int(0.8 * total_samples)
val_end = int(0.9 * total_samples)

In [ ]:
import numpy as np
import chess
from tensorflow.keras.utils import Sequence

In [ ]:
class ChessDataGenerator(Sequence):
    def __init__(self, data, batch_size=32, shuffle=True):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.data))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.data[i] for i in indices]
        X, y = self.__data_generation(batch)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, batch):
        X = np.empty((self.batch_size, 8, 8, 12))
        y = np.empty((self.batch_size), dtype=float)

        for i, item in enumerate(batch):
            X[i,] = self.fen_to_matrix(item[0])
            y[i] = item[1] / 1000.0

        return X, y

    def fen_to_matrix(self, fen):
        piece_map = {
            'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
            'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
        }
        board = chess.Board(fen)
        matrix = np.zeros((8, 8, 12), dtype=int)
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                row, col = divmod(square, 8)
                matrix[row, col, piece_map[piece.symbol()]] = 1
        return matrix

In [ ]:
train_data = data[:train_end]
val_data = data[train_end:val_end]
test_data = data[val_end:]

train_generator = ChessDataGenerator(train_data, batch_size=64, shuffle=True)
val_generator = ChessDataGenerator(val_data, batch_size=64, shuffle=False)
test_generator = ChessDataGenerator(test_data, batch_size=64, shuffle=False)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

In [ ]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(8, 8, 12)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='linear')  # Assuming we're predicting a single evaluation score
])

In [ ]:
# Step 5: Compile the Model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
# Train the Model
history = model.fit(train_generator, epochs=15, validation_data=val_generator)

Epoch 1/15
208496/208496 [==============================] - 2864s 14ms/step - loss: 361.7831 - mae: 4.7840 - val_loss: 362.8983 - val_mae: 5.0444
Epoch 2/15
208496/208496 [==============================] - 2868s 14ms/step - loss: 349.2490 - mae: 4.7777 - val_loss: 356.3472 - val_mae: 4.7903
Epoch 3/15
208496/208496 [==============================] - 2862s 14ms/step - loss: 344.4936 - mae: 4.7557 - val_loss: 353.3282 - val_mae: 4.7591
Epoch 4/15
208496/208496 [==============================] - 2852s 14ms/step - loss: 341.4751 - mae: 4.7437 - val_loss: 351.1648 - val_mae: 4.7946
Epoch 5/15
208496/208496 [==============================] - 2848s 14ms/step - loss: 339.5376 - mae: 4.7404 - val_loss: 350.3852 - val_mae: 4.8036
Epoch 6/15
208496/208496 [==============================] - 2901s 14ms/step - loss: 338.1156 - mae: 4.7362 - val_loss: 349.7501 - val_mae: 4.7642
Epoch 7/15
208496/208496 [==============================] - 2867s 14ms/step - loss: 337.0548 - mae: 4.7325 - val_loss: 348.5

In [ ]:
# Evaluate the Model
test_loss, test_mae = model.evaluate(test_generator)
print(f"Test MAE: {test_mae}")

26062/26062 [==============================] - 313s 12ms/step - loss: 354.2274 - mae: 5.0898
Test MAE: 5.089754104614258


In [ ]:
model.save('/content/drive/MyDrive/saved_models/chess_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
